# 클래스 이름 한글 추가

In [ ]:
import pandas as pd

# class_names 딕셔너리 정의
class_names = {
    0: "계좌번호(손글씨)", 1: "임신출산 진료비 지급 신청서", 2: "자동차 계기판", 3: "입퇴원 확인서", 4: "진단서", 
    5: "운전면허증", 6: "진료비영수증", 7: "통원 진료 확인서", 8: "주민등록증", 9: "여권", 
    10: "진료비 납입 확인서", 11: "약제비 영수증", 12: "처방전", 13: "이력서", 14: "소견서", 
    15: "자동차 등록증", 16: "자동차 번호판"
}

# CSV 파일 경로
meta_csv_file = 'data/meta.csv'

# CSV 파일 읽기
df = pd.read_csv(meta_csv_file)

# 한글 클래스 이름 열 추가
df['class_name_kor'] = df['target'].map(class_names)

# 수정된 CSV 파일 저장
output_csv_file = 'data/meta_kor.csv'
df.to_csv(output_csv_file, index=False)

print("한글 클래스 이름이 추가된 CSV 파일이 저장되었습니다.")

# Train 이미지 폴더별로 나누기

In [ ]:
import os
import pandas as pd
from shutil import copy2

# 파일 경로 설정
csv_file = 'data/train.csv'
meta_csv_file = 'data/meta_kor.csv'
image_folder = 'data/train'  # 이미지 파일들이 있는 폴더
output_folder = 'data/train_by_class'  # 클래스별 폴더를 생성할 위치

# CSV 파일 읽기
train_df = pd.read_csv(csv_file)

# meta_csv 파일 읽기
meta_df = pd.read_csv(meta_csv_file)

# target을 기준으로 class_name_kor과 합치기
train_df = train_df.merge(meta_df[['target', 'class_name_kor']], on='target', how='left')

# 각 클래스별 폴더 생성 및 이미지 이동
for index, row in train_df.iterrows():
    image_id = row['ID']
    class_id = row['target']
    class_name_kor = row['class_name_kor']
    
    # 클래스별 폴더 이름 설정
    class_folder_name = f"{class_id}_{class_name_kor}"
    class_folder = os.path.join(output_folder, class_folder_name)
    
    # 클래스별 폴더가 없으면 생성
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)
    
    # 이미지 파일 경로 설정
    src_image_path = os.path.join(image_folder, image_id)
    dest_image_path = os.path.join(class_folder, image_id)
    
    # 이미지 파일을 클래스 폴더로 복사
    copy2(src_image_path, dest_image_path)

print("이미지 분류가 완료되었습니다.")

# Train Data 라벨링 오류 수정

In [ ]:
import pandas as pd

# 파일 경로 설정
csv_file = 'data/train.csv'
new_csv_file = 'data/train_fixed.csv'

# CSV 파일 읽기
train_df = pd.read_csv(csv_file)

train_df['target'][train_df['ID'] == '45f0d2dfc7e47c03.jpg'] = 7 # 입퇴원확인서->진료확인서
train_df['target'][train_df['ID'] == 'aec62dced7af97cd.jpg'] = 14 # 3 입퇴원확인서->소견서
train_df['target'][train_df['ID'] == '8646f2c3280a4f49.jpg'] = 3 # 7 통원진료확인서->입퇴원확인서
train_df['target'][train_df['ID'] == '1ec14a14bbe633db.jpg'] = 7 # 14 소견서->진료확인서
train_df['target'][train_df['ID'] == '7100c5c67aecadc5.jpg'] = 7 # 3 입퇴원확인서->진료확인서
train_df['target'][train_df['ID'] == 'c5182ab809478f12.jpg'] = 14 # 4 진단서->소견서
train_df['target'][train_df['ID'] == '38d1796b6ad99ddd.jpg'] = 10 # 11 약제비 영수증 -> 진료비(약제비) 납입 확인서
train_df['target'][train_df['ID'] == '0583254a73b48ece.jpg'] = 10 # 11 약제비 영수증 -> 진료비(약제비) 납입 확인서
train_df['target'][train_df['ID'] == '02ebb92c43006832.jpg'] = 10 # 11 약제비 영수증 -> 진료비(약제비) 납입 확인서

# 새 파일 저장
train_df.to_csv(new_csv_file, index=False)

In [ ]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv('data/train_augmented_balanced_150.csv')

# 수정하고자 하는 파일들과 새로운 target 값들을 딕셔너리로 정의
files_to_update = {
    '45f0d2dfc7e47c03.jpg': 7,
    'aec62dced7af97cd.jpg': 14,
    '8646f2c3280a4f49.jpg': 3,
    '1ec14a14bbe633db.jpg': 7,
    '7100c5c67aecadc5.jpg': 7,
    'c5182ab809478f12.jpg': 14,
    '38d1796b6ad99ddd.jpg': 10,
    '0583254a73b48ece.jpg': 10,
    '02ebb92c43006832.jpg': 10,
}

total_updated = 0

for original_file, new_target in files_to_update.items():
    # 원본 파일의 target 변경
    original_updated = df.loc[df['ID'] == original_file, 'target'].shape[0]
    df.loc[df['ID'] == original_file, 'target'] = new_target

    # 증강 파일의 target 변경
    augmented_files = df[df['ID'].str.startswith(original_file.split('.')[0])]
    df.loc[augmented_files.index, 'target'] = new_target

    total_updated += original_updated + len(augmented_files)
    print(f"Updated {original_file} and its {len(augmented_files)} augmented files.")

# 변경된 내용 저장
df.to_csv('data/train_fixed_augmented_balanced_150.csv', index=False)

print(f"Total updated entries: {total_updated}")

# 이미지 개선

In [ ]:
import os
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import multiprocessing
from functools import partial
from shutil import rmtree

def enhance_image(image, gray_scale=False):
    try:
        if gray_scale:
            # 이미지 채널 수 확인
            if len(image.shape) == 2:
                # 이미 그레이스케일
                image = image
            elif len(image.shape) == 3:
                # 컬러 이미지를 그레이스케일로 변환
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                raise ValueError("Unexpected image format")
        
        # 언샤프 마스킹을 사용한 샤프닝
        blurred = cv2.GaussianBlur(image, (0, 0), 4)
        result = cv2.addWeighted(image, 1.55, blurred, -0.5, 0)

        # 최종 결과를 3채널로 변환
        if gray_scale:
            result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)
        
        return result
    except Exception as e:
        print(f"Error in enhance_image: {e}")
        return image  # 오류 발생 시 원본 이미지 반환
    
def process_chunk(chunk, input_dir, output_dir, gray_scale=False):
    processed_images = []
    enhanced_files = []
    
    for filename in chunk:
        img_path = os.path.join(input_dir, filename)
        if gray_scale :
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        else : 
            image = cv2.imread(img_path)

        if image is not None:
            enhanced = enhance_image(image, gray_scale)
            
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, enhanced)
            processed_images.append((filename, enhanced))
    
    return enhanced_files, processed_images

def process(input_dir, output_dir, gray_scale=False, num_processes=multiprocessing.cpu_count()):
    if os.path.exists(output_dir):
        rmtree(output_dir)
    os.makedirs(output_dir)
    
    filenames = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    chunk_size = len(filenames) // num_processes + 1
    chunks = [filenames[i:i+chunk_size] for i in range(0, len(filenames), chunk_size)]
    
    with multiprocessing.Pool(num_processes) as pool:
        results = list(tqdm(
            pool.imap(partial(process_chunk, input_dir=input_dir, output_dir=output_dir, gray_scale=gray_scale), chunks),
            total=len(chunks),
            desc="Processing image chunks"
        ))
    
    enhanced_files = [file for sublist in results for file in sublist[0]]
    processed_images = [img for sublist in results for img in sublist[1]]
    
    print(f"Processed {len(processed_images)} images saved to {output_dir}")
    return enhanced_files, processed_images

def show_sample_images(original_images, processed_images, num_samples=3):
    sample_indices = random.sample(range(len(original_images)), num_samples)
    
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 4*num_samples))
    for i, idx in enumerate(sample_indices):
        original_filename, original_image = original_images[idx]
        _, processed_image = processed_images[idx]
        
        axes[i, 0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        axes[i, 0].set_title(f"Original: {original_filename}")
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB))
        axes[i, 1].set_title(f"Processed: {original_filename}")
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

def enhance(input_dir, output_dir, gray_scale=False):
    enhanced_files, processed_images = process(input_dir, output_dir, gray_scale)

    print("Enhanced files:")
    for filename in enhanced_files:
        print(filename)

    # 원본 이미지 로드 (샘플 이미지 표시를 위해)
    original_images = [(f, cv2.imread(os.path.join(input_dir, f))) for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # 샘플 이미지 표시
    show_sample_images(original_images, processed_images, 3)

enhance('data/train', 'data/train_enhanced', False)
enhance('data/test', 'data/test_enhanced', False)
enhance('data/train', 'data/train_enhanced_gray', True)
enhance('data/test', 'data/test_enhanced_gray', True)

# Train 이미지 오프라인 증강

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import augraphy
from augraphy import AugraphyPipeline
import albumentations as A
from tqdm import tqdm
import cv2
import random
from shutil import rmtree

image_width = 224
image_height = 224

document_classes = [1, 3, 4, 6, 7, 10, 12, 13, 14] # 11 : 약제비 영수증?

class SaltAndPepper(A.ImageOnlyTransform):
    '''
    salt_prob는 흰색 픽셀(소금)이 추가될 확률입니다.
    pepper_prob는 검은색 픽셀(후추)이 추가될 확률입니다.
    p는 이 변환이 적용될 전체 확률입니다
    '''
    def __init__(self, salt_prob=0.01, pepper_prob=0.01, p=0.5):
        super().__init__(p)
        self.salt_prob = salt_prob
        self.pepper_prob = pepper_prob

    def apply(self, image, **params):
        noise = np.zeros(image.shape, np.uint8)
        salt = np.random.random(image.shape) < self.salt_prob
        pepper = np.random.random(image.shape) < self.pepper_prob
        noise[salt] = 255
        noise[pepper] = 0
        return cv2.add(image, noise)

def overlay_images(image1, image2, alpha_image1):
    # 이미지 크기 맞추기
    h1, w1 = image1.shape[:2]
    h2, w2 = image2.shape[:2]
    
    # 크기가 다르면 image2를 image1 크기에 맞게 리사이즈
    if h1 != h2 or w1 != w2:
        image2 = cv2.resize(image2, (w1, h1))
    
    # 채널 수 확인 및 맞추기
    if len(image1.shape) != len(image2.shape):
        if len(image1.shape) == 2:
            image1 = cv2.cvtColor(image1, cv2.COLOR_GRAY2BGR)
        elif len(image2.shape) == 2:
            image2 = cv2.cvtColor(image2, cv2.COLOR_GRAY2BGR)
    
    # 이미지 합성
    return cv2.addWeighted(image1, alpha_image1, image2,1 - alpha_image1, 0)    

def combine_augmentations(image, target, overlay_image=None, overlay_class_image=None):

    # 확률에 따라 한 이미지에 여러 변환이 동시에 적용될 수 있음
    albu_pipeline = A.Compose([
        A.RandomResizedCrop(height=image_height, width=image_width, scale=(0.6, 1.0), p=0.5), # 40% 확률로 적용됨. 이미지를 지정된 크기로 무작위 크롭 및 리사이즈
        A.OneOf([ # 아래 두 변환 중 하나가 70% 확률로 선택되어 적용됨
            A.RandomRotate90(p=0.5),  # 50% 확률로 90도 단위 회전
            A.Rotate(limit=180, p=0.5),  # 50% 확률로 -180~180도 사이 회전
        ], p=0.7),
        A.Flip(p=0.5), # 50% 확률로 이미지를 뒤집음 (수평 또는 수직)
        A.Blur(blur_limit=7, p=0.5),
        A.GaussNoise(var_limit=(10.0, 80.0), p=0.7), # 70% 확률로 가우시안 노이즈 추가
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3), # 30% 확률로 밝기와 대비 무작위 조정
        SaltAndPepper(salt_prob=0.005, pepper_prob=0.01, p=0.2),
    ])    

    augraphy_pipeline = AugraphyPipeline([
        augraphy.DirtyDrum(line_width_range=(1, 2), line_concentration=0.05, p=0.3), # 20% 확률로 더러운 프린터 드럼 효과 적용 (가는 선 추가)
        augraphy.Letterpress(n_samples=(50, 200), n_clusters=(100, 200), std_range=(500, 1500), p=0.3), # 20% 확률로 옛날 인쇄기 효과 적용 (텍스트에 불규칙한 압력 적용)
        augraphy.Markup(num_lines_range=(1, 3), markup_length_range=(0.5, 1.0), p=0.1), # 10% 확률로 수동 마크업 효과 추가 (펜으로 그은 선 등)
        augraphy.NoiseTexturize(sigma_range=(5, 15), turbulence_range=(2, 5), p=0.7),  # 70% 확률로 노이즈 텍스처 추가 (종이의 질감이나 오염 시뮬레이션)
        augraphy.Folding(fold_count=1, fold_angle_range=(0, 30), p=0.2), # 20% 확률로 종이 접힘 효과 추가
    ])
    
    image = albu_pipeline(image=image)['image']

    # 문서 클래스에 대해서만 Augraphy 적용
    if target in document_classes:
        image = augraphy_pipeline(image)
    # 원본 이미지 overlay (5% 확률)
    elif overlay_image is not None and random.random() < 0.05:
        image = overlay_images(image, overlay_image, random.uniform(0.6, 0.8))
    # 다른 클래스 이미지 overlay (5% 확률)
    elif overlay_class_image is not None and random.random() < 0.05:
        image = overlay_images(image, overlay_class_image, random.uniform(0.8, 0.95))

    return image

def augment_and_save(data, all_images, input_dir, output_dir):
    img_name = data['ID']
    target = data['target']
    is_augmented = data['is_augmented']
    
    if is_augmented:
        original_name = img_name.split('_aug_')[0] + os.path.splitext(img_name)[1]
        img_path = os.path.join(input_dir, original_name)
    else:
        img_path = os.path.join(input_dir, img_name)

    img = np.array(Image.open(img_path).convert('RGB'))
    
    if is_augmented:
        other_class_images = [img for img in all_images if img[2] != target]
        if other_class_images:
            overlay_class_img_path = random.choice(other_class_images)[0]
            overlay_class_img = np.array(Image.open(overlay_class_img_path).convert('RGB'))
        else:
            overlay_class_img = None

        augmented = combine_augmentations(img, target, img, overlay_class_img)
    else:
        augmented = img
    
    Image.fromarray(augmented).save(os.path.join(output_dir, img_name))
    
    return data

def process_chunk(chunk, all_images, input_dir, output_dir):
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(partial(augment_and_save, all_images=all_images, input_dir=input_dir, output_dir=output_dir), chunk))
    return results

def process_by_min_class(input_dir, output_dir, input_csv, output_csv, augmentation_ratio, chunk_size=500):
    if os.path.exists(output_dir):
        rmtree(output_dir)
    os.makedirs(output_dir)

    df = pd.read_csv(input_csv)
    
    print(f"시작: 총 {len(df)}개의 원본 이미지 처리 중...")
    
    class_counts = df['target'].value_counts()
    min_count = class_counts.min()
    print(f"min_count : {min_count}")
    target_count = int(min_count * augmentation_ratio)
    print(f"target_count : {target_count}")

    # 각 클래스별 증강해야 할 이미지 수 계산
    class_augment_counts = {target: max(0, target_count - count) for target, count in class_counts.items()}
    print(f"class_augment_counts : {class_augment_counts}")
    
    all_images = [
        (os.path.join(input_dir, row['ID']), row['ID'], row['target'])
        for _, row in df.iterrows()
    ]
    
    all_data = []
    
    # 원본 이미지 추가
    for _, row in df.iterrows():
        all_data.append({
            'ID': row['ID'],
            'target': row['target'],
            'is_augmented': False
        })
    
    # 증강 이미지 생성 및 추가
    for target, augment_count in class_augment_counts.items():
        target_images = [img for img in all_images if img[2] == target]
        for i in range(augment_count):
            original_img = random.choice(target_images)
            aug_name = f"{os.path.splitext(original_img[1])[0]}_aug_{i+1}{os.path.splitext(original_img[1])[1]}"
            all_data.append({
                'ID': aug_name,
                'target': target,
                'is_augmented': True
            })
    
    # 청크 단위로 처리
    for i in tqdm(range(0, len(all_data), chunk_size), desc="Processing chunks"):
        chunk = all_data[i:i+chunk_size]
        process_chunk(chunk, all_images, input_dir, output_dir)
    
    augmented_df = pd.DataFrame(all_data)
    augmented_df.to_csv(output_csv, index=False)
    
    print(f"\n완료: 총 {len(df)}개의 원본 이미지에서 {len(augmented_df)}개의 이미지 데이터 생성 (원본 + 증강)")
    print(f"증강 비율: {len(augmented_df) / len(df):.2f}배")
    
    final_class_counts = augmented_df['target'].value_counts()
    print("\n클래스별 최종 이미지 수:")
    print(final_class_counts)

augmentation_ratio = 150  # 원하는 증강 비율 (예: 1.5배)
chunk_size = 50  # 한 번에 처리할 이미지 수

process_by_min_class('data/train', f'data/train_augmented_balanced_{augmentation_ratio}', 'data/train.csv', f'data/train_augmented_balanced_{augmentation_ratio}.csv', augmentation_ratio, chunk_size)